<a href="https://colab.research.google.com/github/sonjoy2074/CSE4531-Project/blob/main/Federated_Learning/Server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import copy
import torch.nn as nn

from transformers import ViTForImageClassification


# Load local models
local_model_1 = torch.load('/content/drive/MyDrive/Models/doctor_1.pth')
local_model_2 = torch.load('/content/drive/MyDrive/Models/doctor_2.pth')

# Initialize the global model with the same structure
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
n_inputs = model.classifier.in_features
model.classifier = nn.Sequential(
    nn.Linear(n_inputs, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, len(local_model_1['classes']))
)

global_state_dict = copy.deepcopy(local_model_1['model_state_dict'])

# Average the parameters
for key in global_state_dict:
    global_state_dict[key] = (local_model_1['model_state_dict'][key] + local_model_2['model_state_dict'][key]) / 2

# Load averaged weights into the global model
model.load_state_dict(global_state_dict)

# Save the global model
torch.save({
    'model_state_dict': model.state_dict(),
    'classes': local_model_1['classes']
}, '/content/drive/MyDrive/Models/global_model_2.pth')

print("Global model saved as 'global_model.pth'")


<ipython-input-3-00065d101b4a>:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  local_model_1 = torch.load('/content/drive/MyDrive/Models/doctor_1.pth')
<ipython-input-3-000

Global model saved as 'global_model.pth'
